In [5]:
%load_ext line_profiler
%load_ext autoreload
%autoreload 2

In [7]:
import numpy as np
import old.tools, old.RNTK_avg
import jax
import symjax
import symjax.tensor as T
import copy
from RNTK_NEW import RNTK
import time

from RNTK_onediag_dual import RNTK
from RNTK_onediag_dual import create_func as create_func_dual

In [8]:
import numpy as np

# n1 = 10
# n2 = 20

# Qx = np.ones(n1)
# Qxprime = np.ones(n2)
# M = np.ones((n1,n2))


# F,G =  VT(Qx,Qxprime,M)

In [ ]:
# get old vt for the first algorithm

In [12]:
dic = {}
dic["n_patrons1="] = 10#00 #// N (what affect symjax time)
dic["n_entradasTiP="] = 2#00 #// TiPrime length
dic["n_entradasTi="] = 5#00 #// Ti length (multiplier on symjax time)
N = int(dic["n_patrons1="])
ti_length = int(dic["n_entradasTi="])
DATA = T.Placeholder((N, ti_length), 'float32', name = "X")

#testing
DATA_NP = np.zeros((N, ti_length))
DATA_NP_T = np.transpose(DATA_NP)


In [10]:
rntk = RNTK(None, None, None, True)

In [ ]:
# ok so we have to compute for both T and Tprime at once? but does this mean that they have to be the same length? or was this the T + TP?
def compute_q(T, DATA):
    DATAT = T.transpose(DATA)
    xz = DATAT[0]
    init = rntk.su * 2 * T.linalg.norm(xz, ord = 2) + rntk.sb**2 + rntk.sh**2

    def scan_func(MINIDATAT, prevq):
        # the trick to this one is to use the original VT
        S, _ = rntk.alg1_VT(prevq) # -> M is K3 
        newq = rntk.sw*2 * S + rntk.su * 2 * T.linalg.norm(MINIDATAT, ord = 2) + rntk.sb**2
        return newq, newq
    
    T.scan(scan_func, init = init, sequences = DATAT[1:])